# IBM Data Science Capstone Project Notebook

## Project Title: Alternative areas to Asheville, NC

### Introduction / Business Problem

Asheville, North Carolina has been a tourist destination since the middle of the 19th century when people came to the area seeking relief from tuberculosis. At the time it was thought the climate promoted healing and relief of tuberculosis and numerous boarding houses and sanitariums were established to support this industry. In more recent years, Asheville has seen double-digit population growth since the 1990s and is commonly featured on lists of top destinations. Because of this continued growth and demand the cost of housing in Asheville is higher than most other areas in North Carolina. 

The objective of this project is to compare Asheville with other metropolitan areas in North Carolina to identify similar areas in terms of businesses and venues but with lower cost housing. There are factors that we are not considering that can be considered unique to Asheville which could ultimately influence a buyer's decision such as the geography and climate.

The audience for our project is people who would like to purchase a single-family home in Asheville but either can’t afford the cost or choose not to pay the prices. We are attempting to help these people find alternative locations in North Carolina that are similar in terms of businesses/venues to Asheville but with lower cost housing.

### Data

#### House Price Data

We will use the Federal Housing Finance Agency’s House Price Index (HPI) to determine how the price of housing compares across US Census Bureau Metropolitan Statistical Areas (MSAs). We’ll use the most recent HPI data available which is from the first quarter of 2021. We’ll also use the “all-transactions” type which includes both refinance mortgages and purchase-only data. We’re choosing this type because it appears to be the only type for which recent data is available that includes Asheville, NC. We’ll use the non-seasonally adjusted index since it appears the seasonally adjusted index is not available for the “all-transactions” type.

https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index.aspx

NOTE: the HPI focuses on single-family house prices and primarily on mortgages that are purchased and/or securitized by Fannie Mae or Freddie Mac.

There are several MSAs listed for North Carolina, but we will be focusing our comparison on the following MSAs: Asheville, Charlotte-Concord-Gastonia, Durham-Chapel Hill, Greensboro-High Point, Hickory-Lenoir-Morganton, Raleigh-Cary, Wilmington, and Winston-Salem.

#### Location / Venue Data

We will use the Foursquare API to acquire location/venue data by zip-code for comparison.

https://developer.foursquare.com/docs/places-api/

We will use the US Department of Housing and Urban Development’s (HUD) crosswalk between MSAs and zip-codes to acquire the list of zip-codes within each MSA. NOTE: zip-codes can sometimes be located partially inside and outside of an MSA.

https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data
